# 2F2L on Federated Foundation Models (DomainNet as example)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nanboy-Ronan/2F2L-Federated-Unlearning/blob/main/notebooks/domainnet.ipynb) 

This notebook demos how 2F2L works on federated foundation models, where we take the DomainNet as an example and implementes the FL-LP following the paper. For direct observation of the effect of unlearning, we adapt backdoor attacks as describted in our paper.

In [ ]:
!nvidia-smi

Fri Sep 20 04:46:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   47C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Clone the Repo

In [ ]:
!git clone https://github.com/Nanboy-Ronan/2F2L-Federated-Unlearning.git
%cd 2F2L-Federated-Unlearning

Cloning into '2F2L-Federated-Unlearning'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 51 (delta 12), reused 46 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 243.02 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/2F2L-Federated-Unlearning


## Download data and pre-trained checkpoints

This dataset and pretrained checkpoint contains all pre-processed datasets and pre-trained models in our paper.

In [ ]:
!wget -nc https://object-arbutus.cloud.computecanada.ca:443/rjin/2F2L/data.zip

--2024-09-20 04:46:56--  https://object-arbutus.cloud.computecanada.ca/rjin/2F2L/data.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16492177175 (15G) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  15.36G   136MB/s    in 2m 0s   

2024-09-20 04:48:56 (131 MB/s) - ‘data.zip’ saved [16492177175/16492177175]



In [ ]:
!wget -nc https://object-arbutus.cloud.computecanada.ca:443/rjin/2F2L/checkpoint.zip

--2024-09-20 04:48:56--  https://object-arbutus.cloud.computecanada.ca/rjin/2F2L/checkpoint.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3811648030 (3.5G) [application/zip]
Saving to: ‘checkpoint.zip’

checkpoint.zip      100%[===================>]   3.55G   136MB/s    in 28s     

2024-09-20 04:49:25 (129 MB/s) - ‘checkpoint.zip’ saved [3811648030/3811648030]



## Setup the environment

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install salesforce-lavis

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vrhtrqwz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-vrhtrqwz
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 456.8 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [ ]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.11.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
braceexpand==0.1.7
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.7.4
cffi==1.16.0
cfgv==3.4.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
clip @ git+https://github.com/openai/CLIP.git@dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==

## Unzip Data and Checkpoint

In [ ]:
!unzip data.zip
%rm -f data.zip

Archive:  data.zip
   creating: data/
  inflating: data/Domainnet.pth      
   creating: data/FashionMNIST/
   creating: data/FashionMNIST/raw/
  inflating: data/FashionMNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz  
  inflating: data/FashionMNIST/raw/t10k-labels-idx1-ubyte  
 extracting: data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz  
  inflating: data/FashionMNIST/raw/train-images-idx3-ubyte  
  inflating: data/FashionMNIST/raw/train-images-idx3-ubyte.gz  
  inflating: data/FashionMNIST/raw/train-labels-idx1-ubyte  
 extracting: data/FashionMNIST/raw/train-labels-idx1-ubyte.gz  
   creating: data/MNIST/
   creating: data/MNIST/raw/
  inflating: data/MNIST/raw/t10k-images-idx3-ubyte  
  inflating: data/MNIST/raw/t10k-images-idx3-ubyte.gz  
  inflating: data/MNIST/raw/t10k-labels-idx1-ubyte  
 extracting: data/MNIST/raw/t10k-labels-idx1-ubyte.gz  
  inflating: data/MNIST/raw/train-images-idx3-ubyte  
  inflating: data/MNIST/raw/train-

In [ ]:
!unzip checkpoint.zip
%rm -f checkpoint.zip

Archive:  checkpoint.zip
   creating: checkpoint/warmup_ce/
   creating: checkpoint/warmup_ce/mnist/
  inflating: checkpoint/warmup_ce/mnist/bs_32_lr_0.001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/cifar10/
  inflating: checkpoint/warmup_ce/cifar10/bs_32_lr_0.001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/20clsimgnet_clipvit/
  inflating: checkpoint/warmup_ce/20clsimgnet_clipvit/bs_32_lr_0.0001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/domainnet_clipvit/
  inflating: checkpoint/warmup_ce/domainnet_clipvit/bs_32_lr_0.0001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  
   creating: checkpoint/warmup_ce/flowers_blip2/
  inflating: checkpoint/warmup_ce/flowers_blip2/bs_32_lr_0.001_seed_1_rate_0.1_maxrate_0.1_clients5_warmup_epoch_50  


## Federated Learning (FLT)

Due to the constraint of memory and time on this notebook GPU, we skip this FLT step but uses our pre-trained FL checkpoint to perform unleanring. However, you can choose to do FL by following the command below.

In [9]:
!python flt.py --warmup_lr 0.0001 --lr 0.0001 --data_dir ./data --server_rate 0.1 --eval_from 50 --train_epochs 200 --dataset domainnet_clipvit --log --mode mixlinear --remove_idx 0 --num_users 5

2024-09-20 05:59:58.741107: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 05:59:58.791230: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 05:59:58.791277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 05:59:58.792654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 05:59:58.800057: I tensorflow/core/platform/cpu_feature_guar

The test acc is around 96% and the backdoor success rate is around 79%.

## Federated Unlearning (FedRemoval)

In [10]:
!python fedremoval.py --warmup_lr 0.0001 --lr 0.0001 --weight_decay 0.01 --data_dir ./data --server_rate 0.1 --eval_from 50 --num_epochs 400 --train_epochs 200 --removal_lr 0.01 --dataset domainnet_clipvit --log --mode mixlinear --remove_idx 0 --num_users 5 --mu 0

2024-09-20 10:25:45.226063: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 10:25:45.278925: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 10:25:45.278981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 10:25:45.280418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-20 10:25:45.288518: I tensorflow/core/platform/cpu_feature_guar

After unlearning, the test accuracy is 90%, very close to the one before unlearning (90%). THe backdoor sccuess rate decreases to 6%, compared to previously 79%.